# Do not run!!

In [2]:
from modules import components
from unidecode import unidecode
import pandas as pd
import re

In [12]:
#returns dict of all charachter permutations(tuples) and their counts
def permute(l):
    d = {}
    for i in range(len(l)):
        for j in range(i, len(l)):
            if(l[i] != l[j]):
                if((l[i], l[j]) in d.keys()):
                    d[(l[i], l[j])] += 1
                elif ((l[j], l[i]) in d.keys()):
                    d[(l[j], l[i])] += 1
                else:
                    d[(l[i], l[j])] = 1
    return(d)

l = ['a', 'b', 'v', 'abs']
d = permute(l)
# for key in d:
#     print(key[1])
d

{('a', 'b'): 1,
 ('a', 'v'): 1,
 ('a', 'abs'): 1,
 ('b', 'v'): 1,
 ('b', 'abs'): 1,
 ('v', 'abs'): 1}

In [3]:
df = pd.read_csv("../data/csv/c_b_t.csv", encoding='utf-8')
df.head()

,Chapter,Book,Text
0,Invocation,Book,"Praise to Válmíki,(2)bird of charming song,(3)..."
1,Canto I. Nárad.,Book I.,"OM.(8) To sainted Nárad, prince of those Wh..."
2,Canto II. Brahmá's Visit,Book I.,"Válmíki, graceful speaker, heard, To highest..."
3,Canto III. The Argument.,Book I.,The hermit thus with watchful heed Received ...
4,Canto IV. The Rhapsodists.,Book I.,"When to the end the tale was brought, Rose i..."


In [4]:
ch1 = []
ch2 = []
weight = []
canto =[] 
book = []


ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

# file = open("../data/txt/d.txt", 'r', encoding = 'utf-8')
# txt = file.readlines()

# df = pd.read_csv("../data/csv/c_b_t.csv", encoding='utf-8')

window = 30 #of words(0 indexed, iteration, maximum non inclusive)


for idx, row in df.iterrows():
    raw = row["Text"]
    txt = re.sub("'s", "", raw)
    txt = raw.strip().split() #list of words in canto
    
    i = 0
    char_relations = {}
    while (i+window < len(txt)):
        context_list = txt[i:i + window]
        chars = [unidecode(x.lower()) for x in context_list if unidecode(x.lower()) in characters] #list of ppl in context
        #permute all in chars, without mirroring ie., (a, b) = (b, a)
        d_temp = permute(chars)
        for key in d_temp:
            if key in char_relations:
                char_relations[key] += d_temp[key]
            else:
                char_relations[key] = d_temp[key]
        i += window
    for key in char_relations:# all relations in a canto
        ch1.append(key[0])
        ch2.append(key[1])
        weight.append(char_relations[key])
        canto.append(row['Chapter'])
        book.append(row["Book"])

    



In [5]:
components.boot()
# for key in components.boot():
#     print(int(key[-1]) +1 )

{'BOOK 0': "Praise to Válmíki,(2)bird of charming song,(3)Who mounts on Poesy sublimest spray,And sweetly sings with accent clear and strongRáma, aye Ráma, in his deathless lay.Where breathes the man can listen to the strainThat flows in music from Válmíki tongue,Nor feel his feet the path of bliss attainWhen Ráma glory by the saint is sung!The stream Rámáyan leaves its sacred fountThe whole wide world from sin and stain to free.(4)The Prince of Hermits is the parent mount,The lordly Ráma is the darling sea.Glory to him whose fame is ever bright!Glory to him, Prachetas'(5)holy son!Whose pure lips quaff with ever new delightThe nectar-sea of deeds by Ráma done.Hail, arch-ascetic, pious, good, and kind!Hail, Saint Válmíki, lord of every lore!Hail, holy Hermit, calm and pure of mind!Hail, First of Bards, Válmíki, hail once more!",
 'BOOK 1': 'Canto I. Nárad.(7)OM.(8)To sainted Nárad, prince of thoseWhose lore in words of wisdom flows.Whose constant care and chief delightWere Scripture and

In [6]:
chb1= []
chb2 = []
wtb = []
b = [] #for the by books dataframe

boos = components.boot()

ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]


window = 30 #of words(0 indexed, iteration, maximum non inclusive)


for book_key in boos:
    raw = boos[book_key]
    txt = re.sub("'s", "", raw)
    txt = raw.strip().split() #list of words in canto

    i = 0
    char_relations = {}
    while (i + window < len(txt)):
        context_list = txt[i:i + window]
        chars = [unidecode(x.lower()) for x in context_list if unidecode(x.lower()) in characters] #list of ppl in context
        #permute all in chars, without mirroring ie., (a, b) = (b, a)
        d_temp = permute(chars)
        for key in d_temp:
            if key in char_relations:
                char_relations[key] += d_temp[key]
            else:
                char_relations[key] = d_temp[key]
        i += window
    for key in char_relations:# all relations in a book
        chb1.append(key[0])
        chb2.append(key[1])
        wtb.append(char_relations[key])
        b.append(int(book_key[-1]) + 1)



In [7]:
len(b) #idk yyy :========

1833

In [8]:
# for x in weight:
#     if x > 1:
#         print(x)

In [9]:
len(chb1) == len(chb2)==len(wtb) ==len(b) 

True

In [10]:
freq_occs = pd.DataFrame(list(zip(ch1, ch2, weight, canto, book)), columns=['Source', 'Target', 'Weight', 'Canto', 'Book'])
freq_occs.to_csv("../data/csv/freq_occs.csv", index=False)

In [11]:
freq_occ = pd.DataFrame(list(zip(chb1, chb2, wtb, b)), columns=['Source', 'Target', 'Weight', 'Book'])
freq_occ.to_csv("../data/csv/freq_occs_by_book.csv", index=False)

In [12]:
fo = pd.read_csv("../data/csv/freq_occs_by_book.csv", encoding = 'utf-8')
fo.head()

,Source,Target,Weight,Book
0,valmiki,rama,1,1
1,kaikeyi,lakshman,2,2
2,rama,sringavera,1,2
3,rama,ganga,1,2
4,sringavera,ganga,1,2


In [13]:
fo = pd.read_csv("../data/csv/freq_occs.csv", encoding = 'utf-8')
fo.tail()

,Source,Target,Weight,Canto,Book
1147,satrughna,vibhishan,1,Canto CXXX. The Consecration.,BOOK VI.
1148,sugriva,satrunjay,1,Canto CXXX. The Consecration.,BOOK VI.
1149,bharat,sugriva,2,Canto CXXX. The Consecration.,BOOK VI.
1150,rama,sita,1,Canto CXXX. The Consecration.,BOOK VI.
1151,vijay,vamadeva,1,Canto CXXX. The Consecration.,BOOK VI.


In [14]:
books_list = fo.Book.unique()
bl = [x for x in books_list]
bl

['Book I.', 'BOOK II.', 'BOOK III.', 'BOOK IV.', 'BOOK V.', 'BOOK VI.']

In [15]:
fo["Book"] = fo["Book"].apply(lambda x : bl.index(x) + 1)

In [16]:
fo.tail()

,Source,Target,Weight,Canto,Book
1147,satrughna,vibhishan,1,Canto CXXX. The Consecration.,6
1148,sugriva,satrunjay,1,Canto CXXX. The Consecration.,6
1149,bharat,sugriva,2,Canto CXXX. The Consecration.,6
1150,rama,sita,1,Canto CXXX. The Consecration.,6
1151,vijay,vamadeva,1,Canto CXXX. The Consecration.,6


In [17]:
for idx, row in fo.iterrows():
    for x in books_list:
        if row.Book == x:
             

IndentationError: expected an indented block (753260321.py, line 4)

### Context That Won't Kick Me in the Face

In [ ]:
# read> pick word > context as list of n strings(words) > relations will be repeated
#intially I defined n > list of all chars in contex(hardcoded) > permuted to get all relations

In [ ]:
"""will have to maintain index until context window completely passes
 or i can put in the distance between the two right alongside the weight(+ve for ahead, and -ve for what came 
 prior) and just ignore the relation if the distance is the same
 Here: if d(ch1, ch2) = 5, then in the pass where ch2 comes in and picks up ch1: toggle the sign for the distance
 ie., d(ch1, ch2) = -d(ch2, ch1), where the first ch is the context and the second the target
 Or.. I am thinking you dont need to do this: instead, if you have window as 30; then you can just
   perform a forward looking analysis with 15 words ahead: because the 15 words bfore would already have been
   picked up. And since its all the same chapter anyway the relations get clubbed together anyway
   the only difference from the last hardcoded distances technique here is that, the distance is measured as 
   diffusion from each word
 """

In [4]:
df = pd.read_csv("../data/csv/c_b_t.csv", encoding='utf-8')
df.head()

,Chapter,Book,Text
0,Invocation,Book,"Praise to Válmíki,(2)bird of charming song,(3)..."
1,Canto I. Nárad.,Book I.,"OM.(8) To sainted Nárad, prince of those Wh..."
2,Canto II. Brahmá's Visit,Book I.,"Válmíki, graceful speaker, heard, To highest..."
3,Canto III. The Argument.,Book I.,The hermit thus with watchful heed Received ...
4,Canto IV. The Rhapsodists.,Book I.,"When to the end the tale was brought, Rose i..."


In [10]:
# components.books_to_chaps()
# dict of book to list o chaps in it

In [11]:
# x = "will have to maintain index until context window completely passes"
# l = x.split()
# window = 30
# t_idx = l.index('have')
# context = txt[(t_idx - window/2) : (t_idx + window/2)]
# context
## mod here - slice values are negative : 0 if -ve, else ok : must be some sort of fn no?

In [8]:
# returns relations dict with all relations in txt
def get_relations(txt, characters, window):
    
    relations = {}
    for word in txt:
            word = word.lower()
            if word in characters:
                source = word
                t_idx = txt.index(source)
                context = txt[t_idx : int(t_idx + window/2)]
                con_words = [cha for cha in context if cha in characters]
                # d_temp = permute(chars) #(tuple:count) #cant use funtion becos it permutes all
## Equate distances between words- check that they havent already been included in the count
# ie., d(s, t) != -d(t, s). OR NOT?
                for target in con_words:
                    if(source != target):
                        if ((source, target) in relations.keys()):      
                            relations[(source, target)] += 1
                        elif ((target, source) in relations.keys() ):
                            relations[(target, source)] += 1
                        else:
                            relations[(source, target)] = 1
    return relations

In [54]:
t = "a e i o u b v d a c u i p l n t d"
c = ['a', 'c', 'b','d','e']
get_relations(t, c, 10)

{('a', 'e'): 2, ('b', 'd'): 1, ('d', 'a'): 2, ('d', 'c'): 2}

In [55]:

bnc = components.books_to_chaps()
book_ends = [(bnc[key][len(bnc[key]) - 1]) for key in bnc.keys()] #the last chaptrer in each book

for idx, row in df.iterrows():
    if unidecode(row["Chapter"] ).lower().replace(".", "") == unidecode(book_ends[2]).lower().replace(".", ""):
        print(book_ends[1])


Canto LXXVII. Bharat's Departure.


In [ ]:
ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

In [68]:
raw = df["Text"][0]
txt = re.sub("'s", "", raw)
txt = unidecode(txt).strip().lower().split() #list of words in canto
get_relations(txt, characters, 30)

{('valmiki', 'rama'): 1}

In [103]:
ch1 = []
ch2 = []
wt = []
boo = [] 

# relations must be 1 per book
# therefore all relations(weights) obtained within a local context and withina  broader context of a canto must be added

ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

bnc = components.books_to_chaps()
book_ends = [bnc[key][len(bnc[key]) - 1] for key in bnc.keys()] #the last chaptrer in each book

window = 50 #of words(0 indexed, iteration, maximum non inclusive)(context will be window/2 on either side)
#so, basically 1/2
#taking the words from this df will give canto context also, that the books list wont be able to give
# ie., this wont pick up on the last word in a chapter and the first word in the next
# could be skipped though- but good practice considering most epics have a habit of starting each chap with unrelated stories

# this also means that the last chapter of each book must be identified so that the canto wise relations can be added
b_relations = {}
book = 0
for idx, row in df.iterrows():
    raw = row["Text"]
    txt = re.sub("'s", "", raw)
    txt = unidecode(txt).strip().lower().split() #list of words in canto
    
    if unidecode(row["Chapter"]).lower().replace(".", "") != unidecode(book_ends[book]).lower().replace(".", ""): 
        #will need to add stuff from the last canto
         #need tuple of relations with weights: this is for each bok
        d_temp = get_relations(txt, characters, window) #chapter wise relations
        for key in d_temp:
            if key in b_relations:
                b_relations[key] += d_temp[key]
            else:
                b_relations[key] = d_temp[key]
        
        
    else:
        last_temp = get_relations(txt, characters, window)
        for key in last_temp:
            if key in b_relations:
                b_relations[key] += last_temp[key]
            else:
                b_relations[key] = last_temp[key]
        
        for key in b_relations: # all relations in a canto
            ch1.append(key[0])
            ch2.append(key[1])
            wt.append(b_relations[key])
            boo.append(book + 1)
        book += 1
        b_relations = {}
    

    #find relations for the last chapter
    # relations = get_relations(txt)
    # for key in relations: # all relations in a canto
    #     ch1.append(key[0])
    #     ch2.append(key[1])
    #     wt.append(relations[key])
    #     boo.append(book)
    # book += 1
if(len(ch1) == len(ch2) == len(wt) == len(boo)):
    occs = pd.DataFrame(list(zip(ch1, ch2, wt, boo)), columns=['Source', 'Target', 'Weight', 'Book'])
    occs.to_csv("../data/csv/cooccs_books.csv", index=False)

    


In [13]:
ch1 = []
ch2 = []
wt = []
canto = [] 
boo = []

ch = components.get_char_list()[0]
characters = [unidecode(c.lower()) for c in ch]

window = 40 #of words(0 indexed, iteration, maximum non inclusive)(context will be window/2 on either side)
#so, basically 1/2
for idx, row in df.iterrows():
    c_relations = {}
    raw = row["Text"]
    txt = re.sub("'s", "", raw)
    txt = unidecode(txt).strip().lower().split() #list of words in canto
    
    d_temp = get_relations(txt, characters, window) #chapter wise relations
    for key in d_temp:
        if key in c_relations:
            c_relations[key] += d_temp[key]
        else:
            c_relations[key] = d_temp[key]
                
    for key in c_relations: # all relations in a canto
        ch1.append(key[0])
        ch2.append(key[1])
        wt.append(c_relations[key])
        canto.append(row["Chapter"])
        boo.append(row["Book"])

    #find relations for the last chapter
    # relations = get_relations(txt)
    # for key in relations: # all relations in a canto
    #     ch1.append(key[0])
    #     ch2.append(key[1])
    #     wt.append(relations[key])
    #     boo.append(book)
    # book += 1
if(len(ch1) == len(ch2) == len(wt) == len(canto) == len(boo)):
    occs = pd.DataFrame(list(zip(ch1, ch2, wt, canto, boo)), columns=['Source', 'Target', 'Weight', 'Canto', 'Book'])
    occs.to_csv("../data/csv/cooccs_chaps.csv", index=False)

    


In [16]:
o_df = pd.read_csv("../data/csv/cooccs_chaps.csv")
o_df.shape

(2209, 5)